<a href="https://colab.research.google.com/github/AaronGoldsmith/gpt-util/blob/main/Function_Calling_Weather_%3C%3E_Airtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

Add Airtable and OpenAI packages

In [ ]:
%pip install pyairtable openai

In [13]:
import json
import openai
import requests

The function descriptions provide the names of functions and how the LLM should interact with them.  Function descriptions are a list of objects with keys  `name`, `description`, `parameters`. The name is the name of the function. The description provides the LLM with context for when to call the function.



```python
 {
    "name":        <NAME_OF_FUNCTION>,
    "description": <DESCRIPTION_OF_FUNCTION>,
    "parameters":  <FUNCTION_PARAMETERS>
 }
```

Parameters defines what to values to pass to the function when it's called.


Provide each parameter as as a separate key in the properties object.




In [14]:

function_descriptions = [{
  "name": "add_weather_to_airtable",
  "description": "Useful when you need to save weather details to  to Airtable",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "The city/state  for the weather report."
      },
      "temperature": {
        "type": "string",
        "description": "the temperature in degrees farenheit"
      },
      "weather_summary": {
        "type": "string",
        "description": "Current weather summary at a specified location"
      },
    }
  }
}, {
  "name": "request_forecast",
  "description": "Request the forecast at a specified longitude & latitude",
  "parameters": {
    "type": "object",
    "properties": {
      "lat": {
        "type": "string",
        "description": "the latitude of the queried location"
      },
      "lon": {
        "type": "string",
        "description": "the longitude of the queried location"
      }
    }
  }
}, {
  "name": "reverse_geo_lookup",
  "description":
  "Lookup a latitude and longitude from a city, address, or zipcode",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "the location to perform the reverse geo_lookup on"
      }
    }
  }
}]

## Create the "add weather to Airtable" function


If you haven't obtained your Airtable API key, you can do do so [here](https://airtable.com/create/tokens)


In [6]:
from pyairtable import Table
import datetime

In [7]:
airtable_api_key = os.getenv('AIRTABLE_KEY') or getpass("Airtable API key: ")

Airtable API key: ··········


  Application ID and / Table IDs can be found in the address bar on a given Airtable page
`https://airtable.com`/`appjyAU0r4Lw6Cj6K`/`tblPxw9vYfQUt7g19/`
- The Application ID is prefixed with `"app"`
- The Table ID is prefixed with `"tbl"`

In [10]:
APPLICATION_ID = "appjyAU0r4Lw6Cj6K"
TABLE_ID = "tblPxw9vYfQUt7g19"

In [11]:
table = Table(airtable_api_key, APPLICATION_ID, TABLE_ID)

In [12]:
def add_weather_to_airtable(location, temperature, weather_summary):
  current_date = str(datetime.date.today())
  table.create({
    "Location": location,
    "Date": current_date,
    "Temperature": temperature,
    "Forecast": weather_summary,
  })


## Create the open weather map API call function

In [15]:
owm_api_key = os.getenv("owm-api-key") or getpass("Open Weather Map API key: ")

Open Weather Map API key: ··········


In [16]:
def request_forecast(lat: str, lon: str,):
  query = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={owm_api_key}"

  response = requests.get(query)
  return response.json()

## Create the reverse geo lookup function




In [17]:
from requests.structures import CaseInsensitiveDict

In [18]:
geoapify_key = os.getenv("geo-apify-key") or getpass("GeoAPIfy API key: ")

GeoAPIfy API key: ··········


In [32]:
def reverse_geo_lookup(location, limit=5):
  url = f"https://api.geoapify.com/v1/geocode/search?text={location}&apiKey={geoapify_key}"

  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"

  response = requests.get(url, headers=headers)
  features = response.json()['features']
  results = []

  for result in features[:limit]:
    prop = result["properties"]
    if 'city' in prop:
      results.append({
        "city": prop["city"],
        "state": prop["state"],
        "formatted": prop["formatted"],
        "lat": prop["lat"],
        "lon": prop["lon"],
        "popularity": prop["rank"]["popularity"]
      })

  return results

## Create a reusable completion function

We will pass the function_descriptions as an argument to the chatCompletion function

In [4]:
import openai
import os
from getpass import getpass
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_KEY if OPENAI_KEY else getpass("OpenAI API key: ")

OpenAI API key: ··········


In [20]:
def chatCompletion(messages, function_descriptions):
    response = openai.ChatCompletion.create(model="gpt-4-0613",
                                          messages=messages,
                                          functions=function_descriptions,
                                          function_call="auto")
    return response

###  GPT-4 to callback function

Select the function based on the returned value from GPT-4

In [21]:

def function_call(ai_response):
  function_call = ai_response["choices"][0]["message"]["function_call"]
  function_name = function_call["name"]
  arguments = function_call["arguments"]
  if function_name == "request_forecast":
    lat = eval(arguments).get("lat")
    lon = eval(arguments).get("lon")
    return request_forecast(lat=lat, lon=lon)
  elif function_name == "reverse_geo_lookup":
    location = eval(arguments).get("location")
    return reverse_geo_lookup(location)
  elif function_name == "add_weather_to_airtable":
    location = eval(arguments).get("location")
    temperature = eval(arguments).get("temperature")
    weather_summary = eval(arguments).get("weather_summary")
    return add_weather_to_airtable(location, temperature, weather_summary)
  else:
    return



Create a function that will call the the chatCompletions endpoint until the finish_reason is not "function_call"

In [43]:

def begin_function_calling(query, showMessages=False):
  messages = [{"role": "user", "content": query}]

  response = chatCompletion(messages, function_descriptions)

  while response["choices"][0]["finish_reason"] == "function_call":
    function_response = function_call(response)
    messages.append({
      "role":
      "function",
      "name":
      response["choices"][0]["message"]["function_call"]["name"],
      "content":
      json.dumps(function_response)
    })

    response = chatCompletion(messages, function_descriptions)
    if showMessages:
      print("messages\n", messages)
      print("response\n", response)

  return response


In [55]:
user_query = "What is the temperature in London, England?"

In [56]:
res = begin_function_calling(user_query, showMessages=True)

messages
 [{'role': 'user', 'content': 'What is the temperature in London, England?'}, {'role': 'function', 'name': 'reverse_geo_lookup', 'content': '[{"city": "London", "state": "England", "formatted": "London, ENG, United Kingdom", "lat": 51.5073359, "lon": -0.12765, "popularity": 9.988490181891963}, {"city": "London", "state": "England", "formatted": "Royal Courts of Justice, Strand, London, WC2A 2LL, United Kingdom", "lat": 51.51419575, "lon": -0.11347677488453559, "popularity": 8.988490181891963}, {"city": "London", "state": "England", "formatted": "Apsley House, 149 Piccadilly, London, W1J 7NT, United Kingdom", "lat": 51.5034813, "lon": -0.1517488185897429, "popularity": 8.988490181891963}, {"city": "London", "state": "England", "formatted": "The Waldorf Hilton, London, Aldwych, London, WC2B 4DD, United Kingdom", "lat": 51.5127794, "lon": -0.11905006689937492, "popularity": 8.988490181891963}, {"city": "London", "state": "England", "formatted": "Griffin Sports Ground (King\'s Col

In [50]:
print(res['choices'][0]["message"]["content"])

The current temperature in London, England is approximately 15 degrees Celsius with clear sky.


In [51]:
user_query = "What is the temperature in London, England? Write a short weather report and add it to Airtable."

In [52]:
res = begin_function_calling(user_query, False)

messages
 [{'role': 'user', 'content': 'What is the temperature in London, England? Write a short weather report and add it to Airtable.'}, {'role': 'function', 'name': 'reverse_geo_lookup', 'content': '[{"city": "London", "state": "England", "formatted": "London, ENG, United Kingdom", "lat": 51.5073359, "lon": -0.12765, "popularity": 9.988490181891963}, {"city": "London", "state": "England", "formatted": "Royal Courts of Justice, Strand, London, WC2A 2LL, United Kingdom", "lat": 51.51419575, "lon": -0.11347677488453559, "popularity": 8.988490181891963}, {"city": "London", "state": "England", "formatted": "Apsley House, 149 Piccadilly, London, W1J 7NT, United Kingdom", "lat": 51.5034813, "lon": -0.1517488185897429, "popularity": 8.988490181891963}, {"city": "London", "state": "England", "formatted": "The Waldorf Hilton, London, Aldwych, London, WC2B 4DD, United Kingdom", "lat": 51.5127794, "lon": -0.11905006689937492, "popularity": 8.988490181891963}, {"city": "London", "state": "Engla